In [7]:
# Import pandas and numpy
import pandas as pd
import numpy as np

# Import citypy to get cities form lat and long.
from citipy import citipy

# Import the requests library.
import requests

# Import the weather API key.
from config import weather_api_key

# Import the time library for sleeping between reqeusts.
import time

In [8]:
# Create a set of 2,000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [9]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [10]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

766

In [11]:
# Starting URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [12]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        current_description = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,      
                          "Current Description": current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | new norfolk
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | smithers
Processing Record 5 of Set 1 | ponta do sol
Processing Record 6 of Set 1 | piranshahr
Processing Record 7 of Set 1 | alice springs
Processing Record 8 of Set 1 | carutapera
Processing Record 9 of Set 1 | butaritari
Processing Record 10 of Set 1 | mahebourg
Processing Record 11 of Set 1 | atambua
Processing Record 12 of Set 1 | guaxupe
Processing Record 13 of Set 1 | sao filipe
Processing Record 14 of Set 1 | scottsburgh
City not found. Skipping...
Processing Record 15 of Set 1 | kaitangata
Processing Record 16 of Set 1 | bethel
Processing Record 17 of Set 1 | saint-philippe
Processing Record 18 of Set 1 | jamestown
Processing Record 19 of Set 1 | hobart
Processing Record 20 of Set 1 | kamaishi
Processing Record 21 of Set 1 | yuzhne
Processing Record 22 of Set 1 | 

In [13]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,New Norfolk,AU,-42.7826,147.0587,65.91,53,99,4.85,overcast clouds
1,Rikitea,PF,-23.1203,-134.9692,78.96,71,1,16.71,clear sky
2,Punta Arenas,CL,-53.1500,-70.9167,57.31,47,75,21.85,broken clouds
3,Smithers,CA,54.7804,-127.1743,37.72,56,40,0.00,scattered clouds
4,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,40,5.10,scattered clouds
5,Piranshahr,IR,36.7010,45.1413,25.52,62,32,1.28,scattered clouds
6,Alice Springs,AU,-23.7000,133.8833,63.95,68,0,4.61,clear sky
7,Carutapera,BR,-1.1950,-46.0200,77.18,91,99,5.95,overcast clouds
8,Butaritari,KI,3.0707,172.7902,82.69,70,99,14.74,overcast clouds
9,Mahebourg,MU,-20.4081,57.7000,80.89,83,75,5.75,light intensity shower rain


In [14]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")